# 🏎️ Model 2A : Light GBM - Multi Target Classification


## Step 1 : Import Libraries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import joblib
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    RandomizedSearchCV,
    StratifiedKFold,
    learning_curve 
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    classification_report,
    confusion_matrix,
    make_scorer
)

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print(" All libraries imported successfully!")

 All libraries imported successfully!


## Step 2 : Load Data

In [12]:
print(" Loading processed data...")


train_df = pd.read_parquet('../data/processed/train_data_v2.parquet')
test_df = pd.read_parquet('../data/processed/test_data_v2.parquet')


train_weights = np.load('../data/processed/train_weights.npy')
test_weights = np.load('../data/processed/test_weights.npy')


with open('../data/processed/metadata_v2.json', 'r') as f:
    metadata = json.load(f)

print(f"\n✅ Data loaded successfully!")
print(f"   Training samples: {len(train_df)} (2024 season)")
print(f"   Test samples: {len(test_df)} (2025 season)")
print(f"   Features: {len(metadata['feature_columns'])}")

 Loading processed data...

✅ Data loaded successfully!
   Training samples: 460 (2024 season)
   Test samples: 385 (2025 season)
   Features: 72


## Step 3 : Prepare features and targets

In [13]:
feature_cols = metadata['feature_columns']

classification_targets = ['win', 'podium', 'points_finish', 'top5']

X_train = train_df[feature_cols]
X_test = test_df[feature_cols]

y_train_class = train_df[classification_targets]
y_test_class = test_df[classification_targets]

X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

print(f"\nData prepared!")
print(f"\nClass Imbalance Analysis (Training):")
for target in classification_targets:
    pos_count = train_df[target].sum()
    neg_count = len(train_df) - pos_count
    imbalance_ratio = neg_count / pos_count if pos_count > 0 else 0
    print(f"   {target:15s}: {pos_count:4d} positive ({pos_count/len(train_df)*100:5.1f}%) | "
          f"{neg_count:4d} negative | Ratio: 1:{imbalance_ratio:.1f}")


Data prepared!

Class Imbalance Analysis (Training):
   win            :   24 positive (  5.2%) |  436 negative | Ratio: 1:18.2
   podium         :   72 positive ( 15.7%) |  388 negative | Ratio: 1:5.4
   points_finish  :  234 positive ( 50.9%) |  226 negative | Ratio: 1:1.0
   top5           :  120 positive ( 26.1%) |  340 negative | Ratio: 1:2.8


## Step 4 : Define Evaluate functions

In [14]:
def calculate_classification_metrics(y_true, y_pred, y_prob=None, weights=None):
   
    acc = accuracy_score(y_true, y_pred, sample_weight=weights)
    prec = precision_score(y_true, y_pred, sample_weight=weights, zero_division=0)
    rec = recall_score(y_true, y_pred, sample_weight=weights, zero_division=0)
    f1 = f1_score(y_true, y_pred, sample_weight=weights, zero_division=0)
    auc = roc_auc_score(y_true, y_prob, sample_weight=weights) if y_prob is not None else None
    
    return {
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1,
        'AUC': auc
    }


def find_optimal_threshold(y_true, y_prob, weights=None):
    
    thresholds = np.arange(0.1, 0.9, 0.05)
    best_f1 = 0
    best_threshold = 0.5
    
    for thresh in thresholds:
        y_pred = (y_prob >= thresh).astype(int)
        f1 = f1_score(y_true, y_pred, sample_weight=weights, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = thresh
    
    return best_threshold, best_f1


def plot_learning_curve(model, X, y, weights=None, cv=5, title="Learning Curve"):

    fit_params = {'sample_weight': weights} if weights is not None else None
    
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, 
        train_sizes=np.linspace(0.1, 1.0, 10),
        cv=cv,
        n_jobs=-1,
        scoring='f1' if hasattr(y, 'unique') and len(np.unique(y)) == 2 else 'neg_mean_absolute_error',
        fit_params= fit_params
    )   
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, label='Training score', color='#3671C6', marker='o')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='#3671C6')
    plt.plot(train_sizes, val_mean, label='CV score', color='#E10600', marker='s')
    plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2, color='#E10600')
    plt.xlabel('Training Set Size', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.title(title, fontweight='bold', fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

print("Evaluation functions defined!")

Evaluation functions defined!


## Step 5 : Baseline Classification Model

In [18]:
baseline_class_params = {
    'objective': 'binary',        
    'metric': 'binary_logloss',   
    'boosting_type': 'gbdt',
    'n_estimators': 1000,         
    'max_depth': 8,               
    'learning_rate': 0.05,        
    'num_leaves': 20,             
    'min_child_samples': 20,     
    'subsample': 0.8,             
    'colsample_bytree': 0.8,      
    'reg_alpha': 0.1,             
    'reg_lambda': 0.1,            
    'class_weight': 'balanced',   
    'random_state': 42,
    'verbose': -1,
    'n_jobs': -1
}

print("\nConservative Baseline Parameters:")
for param, value in baseline_class_params.items():
    if param not in ['random_state', 'verbose', 'n_jobs', 'objective', 'metric']:
        print(f"   {param}: {value}")

baseline_class_models = {}
baseline_class_results = {}


for target in classification_targets:
    print(f"\n  Training baseline LightGBM classifier for '{target}'...")
    
    model = lgb.LGBMClassifier(**baseline_class_params)
    

    model.fit(
        X_train, y_train_class[target],
        sample_weight=train_weights,
        eval_set=[(X_test, y_test_class[target])],      
        eval_sample_weight=[test_weights],
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)] 
    )
    
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_prob = model.predict_proba(X_train)[:, 1]
    test_prob = model.predict_proba(X_test)[:, 1]
    
    
    optimal_thresh, optimal_f1 = find_optimal_threshold(
        y_test_class[target], test_prob, test_weights
    )
    test_pred_optimized = (test_prob >= optimal_thresh).astype(int)
    
    
    train_metrics = calculate_classification_metrics(
        y_train_class[target], train_pred, train_prob, train_weights
    )
    test_metrics = calculate_classification_metrics(
        y_test_class[target], test_pred, test_prob, test_weights
    )
    test_metrics_optimized = calculate_classification_metrics(
        y_test_class[target], test_pred_optimized, test_prob, test_weights
    )
    
    
    baseline_class_models[target] = model
    baseline_class_results[target] = {
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'test_metrics_optimized': test_metrics_optimized,
        'train_pred': train_pred,
        'test_pred': test_pred,
        'test_pred_optimized': test_pred_optimized,
        'train_prob': train_prob,
        'test_prob': test_prob,
        'optimal_threshold': optimal_thresh,
        'best_iteration': model.best_iteration_
    }
    
    print(f" Results for '{target}':")
    print(f"   Default Threshold (0.50)")
    print(f"      ➤ F1 Score : {test_metrics['F1']:.3f}")
    print(f"      ➤ ROC AUC  : {test_metrics['AUC']:.3f}")
    print(f"   Optimized Threshold ({optimal_thresh:.2f})")
    print(f"      ➤ F1 Score : {test_metrics_optimized['F1']:.3f}")
    print(f"      ➤ ROC AUC  : {test_metrics_optimized['AUC']:.3f}")

    print("-"*80)

print("\n✅ All baseline models trained with early stopping!")


Conservative Baseline Parameters:
   boosting_type: gbdt
   n_estimators: 1000
   max_depth: 8
   learning_rate: 0.05
   num_leaves: 20
   min_child_samples: 20
   subsample: 0.8
   colsample_bytree: 0.8
   reg_alpha: 0.1
   reg_lambda: 0.1
   class_weight: balanced

  Training baseline LightGBM classifier for 'win'...
 Results for 'win':
   Default Threshold (0.50)
      ➤ F1 Score : 0.546
      ➤ ROC AUC  : 0.962
   Optimized Threshold (0.70)
      ➤ F1 Score : 0.572
      ➤ ROC AUC  : 0.962
--------------------------------------------------------------------------------

  Training baseline LightGBM classifier for 'podium'...
 Results for 'podium':
   Default Threshold (0.50)
      ➤ F1 Score : 0.728
      ➤ ROC AUC  : 0.950
   Optimized Threshold (0.70)
      ➤ F1 Score : 0.753
      ➤ ROC AUC  : 0.950
--------------------------------------------------------------------------------

  Training baseline LightGBM classifier for 'points_finish'...
 Results for 'points_finish':
   Def